In [25]:
import pandas as pd

In [27]:
df = pd.read_csv("DataProfanity.csv")

In [28]:
df.count()

preprocessed_text    3322
toxicity             3322
dtype: int64

In [29]:
df.head()

,preprocessed_text,toxicity
0,69,1.0
1,@55,1.0
2,@ssfcker,2.8
3,@ssfucker,2.8
4,@ssfvcker,2.4


In [31]:
unique_counts = df.apply(lambda x: x.nunique())
print(unique_counts)
total_unique_words = unique_counts.sum()
print("Total number of unique words in all columns:", total_unique_words)

preprocessed_text    3322
toxicity               11
dtype: int64
Total number of unique words in all columns: 3333


In [32]:
df.describe()

,toxicity
count,3322.000000
mean,2.043347
std,0.529229
min,1.000000
25%,1.800000
50%,2.000000
75%,2.400000
max,3.000000


In [33]:
df.tail()

,preprocessed_text,toxicity
3317,nudes,2.0
3318,she male,2.0
3319,shibary,2.0
3320,three some,2.0
3321,yury,2.0


In [34]:
import pandas as pd

# Initialize an empty list to store unique profanity words and severity ratings
data = []

# Iterate through the Profanity column
for value in df['Profanity']:
    # Check if Canonical Form 1 exists
    if 'Canonical Form 1' in df.columns:
        canonical_form_1_values = df[df['Profanity'] == value]['Canonical Form 1'].dropna().unique()
        for cf1_value in canonical_form_1_values:
            if ':' in cf1_value:
                profanity_word, severity_rating = cf1_value.split(':')
            else:
                profanity_word = cf1_value.strip()
                severity_rating = str(df[df['Profanity'] == value]['Severity Rating'].iloc[0])  # Convert to string
            # Convert severity_rating to integer, handling non-numeric characters
            severity_rating = ''.join(filter(str.isdigit, severity_rating))
            data.append({'Profanity': profanity_word.strip(), 'Severity Rating': int(severity_rating.strip())})

    # Check if Canonical Form 2 exists
    if 'Canonical Form 2' in df.columns:
        canonical_form_2_values = df[df['Profanity'] == value]['Canonical Form 2'].dropna().unique()
        for cf2_value in canonical_form_2_values:
            if ':' in cf2_value:
                profanity_word, severity_rating = cf2_value.split(':')
            else:
                profanity_word = cf2_value.strip()
                severity_rating = str(df[df['Profanity'] == value]['Severity Rating'].iloc[0])  # Convert to string
            # Convert severity_rating to integer, handling non-numeric characters
            severity_rating = ''.join(filter(str.isdigit, severity_rating))
            data.append({'Profanity': profanity_word.strip(), 'Severity Rating': int(severity_rating.strip())})

    # Check if Canonical Form 3 exists
    if 'Canonical Form 3' in df.columns:
        canonical_form_3_values = df[df['Profanity'] == value]['Canonical Form 3'].dropna().unique()
        for cf3_value in canonical_form_3_values:
            if ':' in cf3_value:
                profanity_word, severity_rating = cf3_value.split(':')
            else:
                profanity_word = cf3_value.strip()
                severity_rating = str(df[df['Profanity'] == value]['Severity Rating'].iloc[0])  # Convert to string
            # Convert severity_rating to integer, handling non-numeric characters
            severity_rating = ''.join(filter(str.isdigit, severity_rating))
            data.append({'Profanity': profanity_word.strip(), 'Severity Rating': int(severity_rating.strip())})

# Convert the list of dictionaries to a DataFrame
new_df = pd.DataFrame(data)

# Drop duplicate rows if any
new_df = new_df.drop_duplicates()

# Reset the index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)

len(new_df)

KeyError: 'Profanity'

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [36]:
new_df.head()

,Profanity,Severity Rating
0,towelhead,30
1,ass,30
2,nigger,30
3,wetback,30
4,spic,30


In [37]:
profanity_words = []
for column in new_df.columns:
    if column != 'Severity Rating':
        profanity_words.extend(new_df[column].dropna().unique())
profanity_words = list(set(profanity_words))
print(profanity_words)

['wound', 'penis', 'queer', 'rectum', 'ramrod', 'man chowder', 'motherfucker', 'choad', 'whack off', 'wank', 'trash', 'redskin', 'cholo', 'porch monkey', 'bang', 'pig', 'nymph', 'fuck', 'ass', 'twat', 'twink', 'nigger', 'gowl', 'pecker', 'jailbait', 'coon', 'dump a load', 'foreskin', 'garbage bag', 'kike', 'dick', 'grope', 'blowjob', 'shlong', 'schlong', 'bastard', 'jiggaboo', 'upskirt', 'spunk', 'transvestite', 'throater', 'klan', 'butt', 'pole sucker', 'ape', 'tit', 'doggy style', 'faggot', 'reacharound', 'peter puffer', 'pull the pud', 'tits', 'slit', 'blacky', 'hooters', 'piss', 'cum', 'meat', 'taking the piss', 'throat yogurt', 'hoe', 'pole smoker', 'douche', 'breast', 'ballsack', 'cock', 'spank', 'salad tosser', 'blow a load', 'dolt', 'funbags', 'anus', 'slant eye', 'mongoloid', 'nipple', 'injun', 'weiner', 'fudge packer', 'fellatio', 'ejaculation', 'hell', 'nigga', 'double pen', 'sphincter', 'bonk', 'scum', 'tickle the pickle', 'tossing salad', 'clit', 'monkey', 'abraham', 'shem

In [38]:
len(profanity_words)

252

In [40]:
# Vectorize profanity words
tfidf_vectorizer = TfidfVectorizer()
profanity_vectors = tfidf_vectorizer.fit_transform(profanity_words)

In [41]:
def detect_profanity(sentence):
    sentence_tokens = sentence.split()
    severity_rating = 0
    profanity_count = 0
    max_severity_rating = 0
    for word in sentence_tokens:
        word_vector = tfidf_vectorizer.transform([word])
        similarity_scores = cosine_similarity(word_vector, profanity_vectors)
        max_similarity_score = np.max(similarity_scores)
        if max_similarity_score > 0.8:  # Adjust threshold
            profanity_count += 1
            profanity_word_index = np.argmax(similarity_scores)
            severity_rating += new_df.loc[profanity_word_index, 'Severity Rating']
            max_severity_rating = max(max_severity_rating, new_df.loc[profanity_word_index, 'Severity Rating'])
    
    # Increase severity if multiple profanities are found
    if profanity_count > 1:
        severity_rating += max_severity_rating * (profanity_count - 1)
    
    # Cap severity rating at 3
    severity_rating = min(severity_rating, 3)
    
    return severity_rating

In [44]:
# Test the function
input_sentence = "black bi"
severity_rating = detect_profanity(input_sentence)
if severity_rating > 0:
    print("Severity Rating:", severity_rating)
else:
    print("The sentence does not contain profanity.")

The sentence does not contain profanity.


In [18]:
input_sentence = "black man"
word_vector = tfidf_vectorizer.transform([input_sentence])
similarity_scores = cosine_similarity(word_vector, profanity_vectors)
max_similarity_score = np.max(similarity_scores)
print(max_similarity_score)

0.681237902243493


In [22]:
sentence = "black man"

In [23]:
# Split the string into a list of words
word_list = sentence.split()

# Print the resulting list
print(word_list)

['black', 'man']


In [24]:
for word in word_list:
    severity_rating = detect_profanity(word)
    rating = str(severity_rating)
    if severity_rating > 0:
        print(f"Word '{word}' has profanity. Severity Rating:", rating)
    else:
        print(f"Word '{word}' does not contain profanity.")

Word 'black' does not contain profanity.
Word 'man' does not contain profanity.
